In [15]:
import sqlite3
import pandas as pd
import MeCab
import ipadic
import re



In [10]:
_DB_PATH = "../ffgcrawl/db.sqlite"
_TAGGER = MeCab.Tagger(ipadic.MECAB_ARGS)

In [11]:
def db_con() -> sqlite3.Connection:
    con: sqlite3.Connection = sqlite3.connect(_DB_PATH)
    con.row_factory = sqlite3.Row
    return con


def query(sql: str) -> pd.DataFrame:
    return pd.read_sql(sql, db_con())

In [12]:
question = "ふくおかフィナンシャルグループのマネーサービスについて教えて下さい"

In [16]:
def extract_keywords(s: str) -> list[str]:
    words = _TAGGER.parse(s).split("\n")
    # 最後の2個はEOSと空文字なので削除
    infos = list(map(lambda s: re.split(r"[\t,]", s), words))[:-2]
    ret: list[str] = []
    for info in infos:
        if info[1] == "名詞" and info[2] in ("一般", "固有名詞"):
            ret.append(info[7])
    return ret

In [17]:
q_kw = extract_keywords(question)

In [20]:
_TAGGER.parse(question).split("\n")

['ふくおか\t名詞,固有名詞,人名,姓,*,*,ふくおか,フクオカ,フクオカ',
 'フィナンシャルグループ\t名詞,一般,*,*,*,*,*',
 'の\t助詞,連体化,*,*,*,*,の,ノ,ノ',
 'マネー\t名詞,一般,*,*,*,*,マネー,マネー,マネー',
 'サービス\t名詞,サ変接続,*,*,*,*,サービス,サービス,サービス',
 'について\t助詞,格助詞,連語,*,*,*,について,ニツイテ,ニツイテ',
 '教え\t動詞,自立,*,*,一段,連用形,教える,オシエ,オシエ',
 'て\t助詞,接続助詞,*,*,*,*,て,テ,テ',
 '下さい\t動詞,非自立,*,*,五段・ラ行特殊,命令ｉ,下さる,クダサイ,クダサイ',
 'EOS',
 '']

In [18]:
q_kw

['ふくおか', '*', 'マネー']